# Explorative Datenanalyse Part 1

## Imports

Der Einfachheit halber regelt config.py alle Imports. Alle Module liegen in einem Package rnaloops \
und sollten nur außerhalb von diesem importiert werden (um relative import Probleme zu vermeiden). \
Wir brauchen nur alle Module aus config.py in einem neuen Notebook zu importieren und haben alles benötigte zur Verfügung. \
User-definierte Funktionen zur EDA liegen z.B. in rnaloops.data_explorer.explore_fcts und werden automatisch importiert.

In [ ]:
from config import *

## Daten laden und prüfen

Daten liegen in rnaloops/data in verschiedenen Formaten. Am besten ist .pkl um gegebenenfalls Metadaten der DatFrames mit zu speichern. \
Es gibt eine Datei mit Daten nach parsing aus den pdfs (rnaloops_data) und eine aufbereitete (rnaloops_data_prepared). Hier ein Vergleich der Spalten:

In [ ]:
df_raw = get_df()  # Fct from getter_fcts.py to load parsed data
df = get_prepared_df()  # Fct from getter_fcts.py to load prepared data 
compare_raw_and_prepared(df_raw, df)  # Fcts to compare them, i.e. RAM use

# Note on string memory reduction:
# Reduction in memory for string type columns due to switch from string to 
# string arrow, which is shown as string by pandas. The difference emerges 
# sice string arrow avoids python overhead on strings (the shorter the strings
# the better the improvement).

# Note on data getting:
# Data is aquired by screening online db in relevant index range and download
# from all pages showing structures. Infos are downloaded as pdf and first
# parsed in plain files, then in a single table. By this we retrieve all
# information on the structure provided by the online db, EXCEPT the first and
# second strand of the conection helices, which (for unknown reasons) is given
# in pdf, but has always value '-'. At his point we dont need that info.
# For more details on the process to get rnaloops_data.pkl check data_getter.py.

Wir sehen nicht alle der 144 Spalten, wo eine 1 im Namen steht gibt es immer 14, eine für jeden (möglichen) Stem. \
Der DataFrame trägt auch eine Funktion um Spalten infos auszugeben, wir können hier später alles relevante ergänzen:

In [ ]:
df.columns_info()

Beim Präparieren wurden Spaltennamen vereinfacht und Dateitypen angepasst um Memory zu sparen (vgl. data_getter.getter_fcts.prepare_df). \
Im Folgenden konzentrieren wir und vor allem auf die numerischen Spalten und ggbf. die kategorischen zur Gruppierung der Daten. \
Wir können diese nochmal genauer angucken:

In [ ]:
df.describe()

Wir haben nur noch 75222 Einträge. Laut Autoren sollen 84261 Strukturen vorliegen, es konnen aber nur knapp 80k heruntergeladen werden. \
Der Rest existiert entweder nicht oder ist sehr gut versteckt ;) ... Außerdem gibt es ca. 2300 Strukturen ohne Infos, die auch entfernt wurden. \
Dazu sind ~3000 Strukturen doppelt vorhanden, die Duplikate wurden auch entfernt.

Spalten die keine Daten enthalten sollten, weil die Struktur weniger als 14 stems hat werden wie folgt behandelt:
- Bei Winkeln wird NaN gesetzt (okay da Winkel ohnehin floats)
- Bei Längen wird -1 gesetzt (NaN nicht sinnvoll bei int, 0 verwirrend weil Strands mit Länge 0 tatsächlich existieren)
- Bei Positionen und Sequenzen wird NA gesetzt (es gibt auch '-', was so in der DB steht und heißt der Wert ist unbekannt?)
- Beim Gruppieren nach loop_type können die Spalten einfach weggelassen werden

## Ein paar Beispiele

Vor den ersten zusammenfassenden Statistiken auf den Daten erstmal ein paar Beispiele für Strukturen. \
Wir können eine einzelne Struktur mittels show_structure() aus explore_fcts als pd.Series erhalten. Dabei haben wir die Optionen:
- indices (int | Iterable): Liste/int = Gewünschte(r) Struktur idx (default: n zufällige Indices, wähle n beliebig)
- web (bool): Zeigt Website der Struktur(en) aus DB in Browser (benötigt driver, siehe unten). Default False.
- pdf (bool): Zeigt pdf der Struktur(en) in Browser pdf Viewer (lädt herunter falls nicht vorhanden). Default False.
- svg (bool): Zeigt svg image der Struktur(en) in Notebook (lädt vorher herunter). Default False.
- keep_files (bool): Ob die heruntergeladenen Dateien behalten werden sollen. Default: False.

Webanzeige der Struktur benötigt einen webdriver. Bsp. chromedriver von https://chromedriver.chromium.org/downloads (i.d.R. V106). \
Der driver muss in $PATH zu finden sein, Funktion ohne Gewähr.

Wir können z.B. Strukturen vergleichen (hier zufällige):

In [ ]:
s1 = show_structure(df, web=False, pdf=False, svg=False, keep_files=False, n=3)
s1.T.head(7)

Oder Strukturen visualisieren:

In [ ]:
_ = show_structure(df, svg=True)  # Braucht Internet, kann etwas dauern

Ein paar exotische Beispiele:

In [ ]:
large = df[df.strand_1_nts > 50]
_ = show_structure(large, svg=True)

In [ ]:
small = df[(df.loop_type == "3-way") # Structure without any stems
         & (df.strand_1_nts + df.strand_2_nts + df.strand_3_nts == 0)]

In [ ]:
_ = show_structure(small, svg=True)

## Übersicht der Daten

Nächster Schritt wäre jetzt Gruppen von Strukturen zu betrachten und Verteilungen der Feature explorativ zu untersuchen. \
Wechsel dazu der Übersichtlichkeit halber in eda_2.ipynb...